# Data
- files in /Users/hkromer/02_PhD/04_Publiations/01_PaperDrafts/2019/01.CharacterizationPaper/He-3data/He3-results.xlsx
    - Each sheet within the Excel file is for a given angle.
    - In green, the XP data.
    - In yellow the reconstructed data.


In [ ]:
%matplotlib inline

In [ ]:
import time
import matplotlib.pyplot as plt
import matplotlib
import pandas as pd
import numpy as np
import datetime
from matplotlib.ticker import AutoMinorLocator
import re

import os
os.environ["PATH"] += os.pathsep + '/usr/local/texlive/2018/bin/x86_64-darwin'
plt.rc('text', usetex=True)
plt.rc('font', weight='bold')
matplotlib.rcParams['mathtext.fontset'] = 'custom'
matplotlib.rcParams['mathtext.rm'] = 'Arial'
matplotlib.rcParams['mathtext.it'] = 'Arial:italic'
matplotlib.rcParams['mathtext.bf'] = 'Arial:bold'
matplotlib.rcParams['mathtext.tt'] = 'Arial'
matplotlib.rcParams['mathtext.cal'] = 'Arial'
matplotlib.rcParams['text.latex.preamble'] = [r'\usepackage{sfmath} \boldmath']

fname = '/Users/hkromer/02_PhD/04_Publiations/01_PaperDrafts/2019/01.CharacterizationPaper/He-3data/He3-results.xlsx'

# each sheet is a different angle
angles = []

fig = plt.figure(figsize=(8, 6))
ax1 = fig.add_subplot(1, 1, 1)
markersize = 80
fs = 20
markers = ['s', 'd', 'o']
colors =['darkred', 'darkblue', 'darkorange']
labels = ['0°', '90°', '180°']

for jj in np.arange(0,3):
    color = colors[jj]
    marker = markers[jj]  
    data = pd.read_excel(fname, sheet_name=jj, skiprows=3)
    data = data[data.columns[[1,2,5,6]]]
    data.columns = ['exp_energy', 'exp_spectrum_norm', 'reconstr_energy', 'reconstr_spectrum_norm']
#     print(data.columns[[1,2,5,6]])

    angles.append(data)
    print(data.head())
    X = data['exp_energy'].values / 1000  # MeV
    _ = plt.scatter(X, data['exp_spectrum_norm'].values, color=color, marker=marker, s=markersize, alpha=0.75, label=labels[jj])

my_handles, my_labels = ax1.get_legend_handles_labels()
leg1 = plt.legend(my_handles, my_labels, loc=(0.05, 0.65), fontsize=fs-4)
leg1.set_title(r"\textbf{Angle}", prop = {'size': fs-4})

labels = ['Convolution', 'Convolution', 'Convolution']

for jj in np.arange(0,3):
    color = colors[jj]
    data = angles[jj]
    X = data['reconstr_energy'].values / 1000  # MeV

    _1 = plt.plot(X, data['reconstr_spectrum_norm'].values, color=color, alpha=0.75, label=labels[jj])
    
    
my_handles, my_labels = ax1.get_legend_handles_labels()
# print(my_handles, my_labels)
# leg1 = plt.legend(my_handles, my_labels, loc=(0.25, 0.65), markerscale=0, fontsize=fs-4)


ax1.grid(b=True, which='major', linestyle='-')
ax1.grid(b=True, which='minor', linestyle='--')

plt.xlim(1.5,3.0)
plt.ylim(0,1.1)

plt.xlabel(r'\textbf{Neutron energy [MeV]}', fontsize=fs)
plt.ylabel(r'\textbf{Normalized neutron spectrum [-]}', fontsize=fs)

ax1.tick_params('x', colors='black', labelsize=fs-2)	
ax1.tick_params('y', colors='black', labelsize=fs-2)	
# x
minor_locator = AutoMinorLocator(2)
ax1.xaxis.set_minor_locator(minor_locator)
# y
minor_locator = AutoMinorLocator(2)
ax1.yaxis.set_minor_locator(minor_locator)    



leg2 = ax1.legend([my_handles[0]], [my_labels[0]], loc=(0.05, 0.55), fontsize=fs-4)
leg2.legendHandles[0].set_color('black')
# leg2.set_title(r"\textbf{Experiment}", prop = {'size': fs-4})
# Manually add the first legend back
ax1.add_artist(leg1)
plt.show()

# Simulation alone

In [ ]:
# import the data for the non-convolved
fname = '/Users/hkromer/02_PhD/04_Publiations/01_PaperDrafts/2019/01.CharacterizationPaper/He-3data/80kV_simulation_spectra.xlsx'

data = pd.read_excel(fname, skiprows=1)
# data = data[data.columns[[1,2,5,6]]]
# data.columns = ['exp_energy', 'exp_spectrum_norm', 'reconstr_energy', 'reconstr_spectrum_norm']
data = data.iloc[0:3,2:]
data = data.T
data = data.reset_index()
data.columns = ['energy', '0_deg', '90_deg', '180_deg']

# normalize and convert to float
for col in ['0_deg', '90_deg', '180_deg']:
    data[col] = data[col].astype(float)
    data[col] = data[col].replace(0, np.nan)
#     print(np.max(data[col]))
    if col == '0_deg': # benoit said 0 and 180 deg values the maximum should be set to 0
        data[col] = data[col].apply(lambda x: x if x != np.max(data[col]) else 0)
        
    if col == '180_deg': # benoit said 0 and 180 deg values the maximum should be set to 0
        data[col] = data[col].apply(lambda x: x if x != np.max(data[col]) else 0)
#     print(np.max(data[col]))
    data[col] = data[col] / np.max(data[col])
    
data.to_csv('/Users/hkromer/02_PhD/04_Publiations/01_PaperDrafts/2019/01.CharacterizationPaper/Figures/neutron_spectrum/neutron_spectrum_simulation.csv')  
# each sheet is a different angle
angles = []

fig = plt.figure(figsize=(8, 6))
ax1 = fig.add_subplot(1, 1, 1)
markersize = 80
fs = 20
markers = ['s', 'd', 'o']
colors =['darkred', 'darkblue', 'darkorange']
labels = ['0°', '90°', '180°']
columns = ['0_deg', '90_deg', '180_deg']

for jj in np.arange(0,3):
    color = colors[jj]
    marker = markers[jj]  
    col = columns[jj]
    data_t = data[['energy', col]]
    data_t = data_t.sort_values(by='energy')
    X = data_t['energy'].values  # MeV
    _ = plt.plot(X, data_t[col].values, color=color, alpha=0.75, label=labels[jj], linewidth=2.5)

leg1 = plt.legend(loc=(0.05, 0.65), fontsize=fs-4)
leg1.set_title(r"\textbf{Angle}", prop = {'size': fs-4})



ax1.grid(b=True, which='major', linestyle='-')
ax1.grid(b=True, which='minor', linestyle='--')

plt.xlim(1.5,3.0)
plt.ylim(0,1.1)

plt.xlabel(r'\textbf{Neutron energy [MeV]}', fontsize=fs)
plt.ylabel(r'\textbf{Normalized neutron spectrum [-]}', fontsize=fs)

ax1.tick_params('x', colors='black', labelsize=fs-2)	
ax1.tick_params('y', colors='black', labelsize=fs-2)	
# x
minor_locator = AutoMinorLocator(2)
ax1.xaxis.set_minor_locator(minor_locator)
# y
minor_locator = AutoMinorLocator(2)
ax1.yaxis.set_minor_locator(minor_locator)    


plt.show()

# Simulation and experiment

In [ ]:
# import the data for the non-convolved
fname = '/Users/hkromer/02_PhD/04_Publiations/01_PaperDrafts/2019/01.CharacterizationPaper/He-3data/80kV_simulation_spectra.xlsx'
fname_experiment = '/Users/hkromer/02_PhD/04_Publiations/01_PaperDrafts/2019/01.CharacterizationPaper/He-3data/He3-results.xlsx'

data = pd.read_excel(fname, skiprows=1)
# data = data[data.columns[[1,2,5,6]]]
# data.columns = ['exp_energy', 'exp_spectrum_norm', 'reconstr_energy', 'reconstr_spectrum_norm']
data = data.iloc[0:3,2:]
data = data.T
data = data.reset_index()
data.columns = ['energy', '0_deg', '90_deg', '180_deg']

# normalize and convert to float
for col in ['0_deg', '90_deg', '180_deg']:
    data[col] = data[col].astype(float)
    data[col] = data[col].replace(0, np.nan)
#     print(np.max(data[col]))
    if col == '0_deg': # benoit said 0 and 180 deg values the maximum should be set to 0
        data[col] = data[col].apply(lambda x: x if x != np.max(data[col]) else 0)
        
    if col == '180_deg': # benoit said 0 and 180 deg values the maximum should be set to 0
        data[col] = data[col].apply(lambda x: x if x != np.max(data[col]) else 0)
#     print(np.max(data[col]))
    data[col] = data[col] / np.max(data[col])
    
# data.to_csv('/Users/hkromer/02_PhD/04_Publiations/01_PaperDrafts/2019/01.CharacterizationPaper/Figures/neutron_spectrum/neutron_spectrum_simulation.csv')  
# each sheet is a different angle
angles = []

fig = plt.figure(figsize=(8, 6))
ax1 = fig.add_subplot(1, 1, 1)
markersize = 80
fs = 20
markers = ['s', 'd', 'o']
colors =['darkred', 'darkblue', 'darkorange']
labels = ['0°', '90°', '180°']
columns = ['0_deg', '90_deg', '180_deg']

for jj in np.arange(0,3):
    color = colors[jj]
    marker = markers[jj]  
    col = columns[jj]
    data_t = data[['energy', col]]
    data_t = data_t.sort_values(by='energy')
    X = data_t['energy'].values  # MeV
    _ = plt.plot(X, data_t[col].values, color=color, alpha=0.75, label=labels[jj], linewidth=2.5)
    
    # experimental data
    color = colors[jj]
    marker = markers[jj]  
    data_experiment = pd.read_excel(fname_experiment, sheet_name=jj, skiprows=3)
    data_experiment = data_experiment[data_experiment.columns[[1,2,5,6]]]
    data_experiment.columns = ['exp_energy', 'exp_spectrum_norm', 'reconstr_energy', 'reconstr_spectrum_norm']
#     print(data.columns[[1,2,5,6]])

    angles.append(data_experiment)
    print(data_experiment.head())
    X = data_experiment['exp_energy'].values / 1000  # MeV
    _ = plt.scatter(X, data_experiment['exp_spectrum_norm'].values, color=color, marker=marker, s=markersize, alpha=0.75, label=labels[jj])

    my_handles, my_labels = ax1.get_legend_handles_labels()
leg1 = plt.legend(my_handles[0:3], my_labels[0:3], loc=(0.05, 0.65), fontsize=fs-4)
leg1.set_title(r"\textbf{Angle}", prop = {'size': fs-4})





ax1.grid(b=True, which='major', linestyle='-')
ax1.grid(b=True, which='minor', linestyle='--')

plt.xlim(1.5,3.0)
plt.ylim(0,1.1)

plt.xlabel(r'\textbf{Neutron energy [MeV]}', fontsize=fs)
plt.ylabel(r'\textbf{Normalized neutron spectrum [-]}', fontsize=fs)

ax1.tick_params('x', colors='black', labelsize=fs-2)	
ax1.tick_params('y', colors='black', labelsize=fs-2)	
# x
minor_locator = AutoMinorLocator(2)
ax1.xaxis.set_minor_locator(minor_locator)
# y
minor_locator = AutoMinorLocator(2)

ax1.yaxis.set_minor_locator(minor_locator)    

leg2 = ax1.legend([my_handles[0]], ['Simulation'], loc=(0.05, 0.55), fontsize=fs-4)
leg2.legendHandles[0].set_color('black')
# leg2.set_title(r"\textbf{Experiment}", prop = {'size': fs-4})
# Manually add the first legend back
ax1.add_artist(leg1)

plt.show()

In [ ]:
col = '180_deg'


In [ ]:
np.max(data[col]) 

In [ ]:
data[col]